Issues:
**Ensure training directory has jpeg files only**
**path enabled selection**
**query pic dynamic**
**number of results**

In [ ]:
#from keras.applications.resnet50 import ResNet50

In [ ]:
#d_model = ResNet50(include_top=False,weights="imagenet",pooling = "avg")
import sys
pathtodb=sys.argv[1]
pathtoquery=sys.argv[2]
number_of_results=int(sys.argv[3])

In [ ]:
#d_model.save("./resnet50.h5")
from tqdm import tqdm
from pathlib import Path

In [ ]:
Path_of_database = Path(pathtodb)
Path_of_query = str(Path(pathtoquery))
features_of_database = "feature_embeddings.h5"
filenames_of_database = "data_set_names.csv"
batch_size = 1
threads=batch_size

In [ ]:
import keras
model=keras.models.load_model("./resnet50.h5")

In [ ]:
model.Trainable=False

In [ ]:
import os
list_of_files = os.listdir(Path_of_database)
list_of_files = [ i for i in list_of_files if ".jpg" in i]
n_files=len(list_of_files)
#print(list_of_files)

In [ ]:
import h5py
file = h5py.File(features_of_database, "w")    
feature_emb = file.create_dataset("features", (len(list_of_files),2048), h5py.h5t.IEEE_F32LE,compression="lzf")

In [ ]:
import cv2
def pre_proc(img):
    return cv2.resize(img,(256,256))
   

In [ ]:
batch_matr = [x for x in range(0,n_files)]
def batch_preprocess(i):
    matrix = cv2.imread(str(Path_of_database / list_of_files[i]))
    matrix = pre_proc(matrix)
    #print("that",matrix)
    global batch_matr
    batch_matr[i]=matrix

In [ ]:
from concurrent.futures import ThreadPoolExecutor
import numpy as np
def parallel_read(begin,end):
    with ThreadPoolExecutor(max_workers=batch_size) as executor:
        for i in range(begin,end+1):
            executor.submit(batch_preprocess,(i))
    return np.array(batch_matr[begin:end+1],dtype='f')

In [ ]:
def read_data(beg,end):
    if end>n_files:
        end=n_files-1
    batch_value=parallel_read(beg,end)
    return batch_value

In [ ]:
#file.close()

In [ ]:
for index_i in tqdm(range(0,n_files//batch_size,batch_size)):
    batch=read_data(index_i,index_i+batch_size-1)
    res=model.predict(batch)
    for i,each in enumerate(res):
        feature_emb[index_i+i,...]=each
file.close()

In [ ]:
def store_filename(l_o_f):
    with open(filenames_of_database,"w+") as f:
        for i,each in enumerate(l_o_f):
            f.write(str(each)+","+str(i)+"\n")

In [ ]:
store_filename(list_of_files)

In [ ]:
import pandas as pd
dataset_filenames = pd.read_csv(filenames_of_database,header=None)
#print(dataset_filenames)

In [ ]:
def query_pic(path):
    img = cv2.imread(path)
    img = pre_proc(img)
    return (model.predict(img[None,...]))

In [ ]:
from scipy.spatial import distance
def calculate_distances(n):
    vector=query_pic(Path_of_query)
    ld = h5py.File(features_of_database,"r")
    values = []
    for each in ld["features"]:
        values.append((1-distance.cosine(each,vector)))
    from operator import itemgetter
    from heapq import nlargest
    return nlargest(n, enumerate(values), itemgetter(1))

In [ ]:
results=calculate_distances(number_of_results)
import matplotlib.pyplot as plt

plt.figure()
fig=plt.figure(figsize=(8, 8))
columns = 1
rows = 6

for i in range(1, columns*rows +1):
    img=plt.imread(str(Path_of_database / dataset_filenames.loc[results[i-1][0]][0]))
    fig.add_subplot(rows, columns, i)
    plt.imshow(img)
plt.show()